In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

pd.pandas.set_option("display.max_columns", None)


In [2]:
dataset_train = pd.read_csv("X_train.csv")

In [3]:
dataset_train.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,12.247694,0.000000,1.00,0.593445,0.566360,0.0,0.5,0.000000,1.000000,1.0,0.25,0.5,0.80,0.500000,0.0,0.0,0.6,0.444444,0.625,0.384615,0.822581,0.0,0.0,0.5,0.363636,0.50,0.000000,1.0,0.666667,0.50,1.0,0.5,0.0,0.666667,0.116708,0.833333,0.094364,0.126168,0.173111,1.0,0.25,1.0,0.666667,0.312253,0.000000,0.0,0.312253,0.0,0.0,0.25,0.0,0.333333,0.5,0.25,0.166667,0.75,0.00,0.2,0.5,0.792994,0.666667,0.2,0.490591,0.666667,1.0,1.0,0.098315,0.000000,0.0,0.0,0.208333,0.0,1.0,0.50,0.5,0.000000,0.454545,1.0,0.5,0.5
1,1462,12.109011,0.000000,0.75,0.598957,0.622527,0.0,0.5,0.333333,1.000000,1.0,0.50,0.5,0.80,0.333333,0.0,0.0,0.6,0.555556,0.625,0.407692,0.870968,0.5,0.0,0.6,0.454545,0.25,0.083721,1.0,0.666667,0.50,1.0,0.5,0.0,0.833333,0.230175,0.666667,0.000000,0.189720,0.260844,1.0,0.25,1.0,0.666667,0.468253,0.000000,0.0,0.468253,0.0,0.0,0.25,0.5,0.500000,0.5,0.50,0.250000,0.75,0.00,0.2,0.5,0.802548,0.666667,0.2,0.209677,0.666667,1.0,1.0,0.275983,0.048518,0.0,0.0,0.000000,0.0,1.0,0.25,0.0,0.735294,0.454545,1.0,0.5,0.5
2,1463,12.317167,0.235294,0.75,0.558854,0.614005,0.0,0.5,0.333333,1.000000,1.0,0.25,0.5,0.95,0.333333,0.0,0.0,0.4,0.444444,0.500,0.107692,0.225806,0.0,0.0,0.5,0.363636,0.50,0.000000,1.0,0.666667,0.75,0.5,0.5,0.0,0.333333,0.197257,0.666667,0.000000,0.064019,0.182139,1.0,1.00,1.0,0.666667,0.326139,0.376477,0.0,0.548792,0.0,0.0,0.50,0.5,0.500000,0.5,0.25,0.250000,0.75,0.25,1.0,0.5,0.678344,0.333333,0.4,0.323925,0.666667,1.0,1.0,0.148876,0.045822,0.0,0.0,0.000000,0.0,1.0,0.50,0.5,0.000000,0.181818,1.0,0.5,0.5
3,1464,11.849398,0.235294,0.75,0.582212,0.524583,0.0,0.5,0.333333,1.000000,1.0,0.25,0.5,0.95,0.333333,0.0,0.0,0.4,0.555556,0.625,0.100000,0.225806,0.0,0.0,0.5,0.363636,0.25,0.015504,1.0,0.666667,0.75,1.0,0.5,0.0,0.333333,0.150125,0.666667,0.000000,0.151402,0.181747,1.0,0.75,1.0,0.666667,0.325285,0.364125,0.0,0.542672,0.0,0.0,0.50,0.5,0.500000,0.5,0.50,0.333333,0.75,0.25,0.4,0.5,0.675159,0.333333,0.4,0.315860,0.666667,1.0,1.0,0.252809,0.048518,0.0,0.0,0.000000,0.0,1.0,0.25,0.5,0.000000,0.454545,1.0,0.5,0.5
4,1465,12.429216,0.588235,0.75,0.317987,0.335596,0.0,0.5,0.333333,0.666667,1.0,0.25,0.5,1.00,0.333333,0.0,0.5,0.6,0.777778,0.500,0.146154,0.322581,0.0,0.0,0.9,0.727273,0.50,0.000000,0.0,0.666667,0.75,0.5,0.5,0.0,0.833333,0.065586,0.666667,0.000000,0.475234,0.251227,1.0,0.75,1.0,0.666667,0.453388,0.000000,0.0,0.453388,0.0,0.0,0.50,0.0,0.333333,0.5,0.50,0.166667,0.75,0.00,0.2,0.5,0.694268,1.000000,0.4,0.340054,0.666667,1.0,1.0,0.000000,0.110512,0.0,0.0,0.250000,0.0,1.0,0.25,0.5,0.000000,0.000000,1.0,0.5,0.5


In [4]:
# for storing the target feature

Y_train = dataset_train["SalePrice"]

In [5]:
Y_train.head()

0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: SalePrice, dtype: float64

In [6]:
nan_features = [feature for feature in dataset_train.columns if dataset_train[feature].isnull().sum()>1]
nan_values = dataset_train[nan_features].isnull().sum()
nan_values

BsmtFullBath    2
BsmtHalfBath    2
dtype: int64

In [7]:
dataset_train["BsmtFullBath"].unique()

array([0.        , 0.33333333, 0.66666667, 1.        ,        nan])

In [8]:
for feature in nan_features:
    mean_value = dataset_train[feature].mean()
    # print(mean_value)
    dataset_train[feature] = dataset_train[feature].fillna(mean_value)

dataset_train[nan_features].isnull().sum()

BsmtFullBath    0
BsmtHalfBath    0
dtype: int64

In [9]:
# for storing the independent feature

X_train = dataset_train.drop(["Id", "SalePrice"], axis = 1)

In [10]:
X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.000000,1.00,0.593445,0.566360,0.0,0.5,0.000000,1.000000,1.0,0.25,0.5,0.80,0.500000,0.0,0.0,0.6,0.444444,0.625,0.384615,0.822581,0.0,0.0,0.5,0.363636,0.50,0.000000,1.0,0.666667,0.50,1.0,0.5,0.0,0.666667,0.116708,0.833333,0.094364,0.126168,0.173111,1.0,0.25,1.0,0.666667,0.312253,0.000000,0.0,0.312253,0.0,0.0,0.25,0.0,0.333333,0.5,0.25,0.166667,0.75,0.00,0.2,0.5,0.792994,0.666667,0.2,0.490591,0.666667,1.0,1.0,0.098315,0.000000,0.0,0.0,0.208333,0.0,1.0,0.50,0.5,0.000000,0.454545,1.0,0.5,0.5
1,0.000000,0.75,0.598957,0.622527,0.0,0.5,0.333333,1.000000,1.0,0.50,0.5,0.80,0.333333,0.0,0.0,0.6,0.555556,0.625,0.407692,0.870968,0.5,0.0,0.6,0.454545,0.25,0.083721,1.0,0.666667,0.50,1.0,0.5,0.0,0.833333,0.230175,0.666667,0.000000,0.189720,0.260844,1.0,0.25,1.0,0.666667,0.468253,0.000000,0.0,0.468253,0.0,0.0,0.25,0.5,0.500000,0.5,0.50,0.250000,0.75,0.00,0.2,0.5,0.802548,0.666667,0.2,0.209677,0.666667,1.0,1.0,0.275983,0.048518,0.0,0.0,0.000000,0.0,1.0,0.25,0.0,0.735294,0.454545,1.0,0.5,0.5
2,0.235294,0.75,0.558854,0.614005,0.0,0.5,0.333333,1.000000,1.0,0.25,0.5,0.95,0.333333,0.0,0.0,0.4,0.444444,0.500,0.107692,0.225806,0.0,0.0,0.5,0.363636,0.50,0.000000,1.0,0.666667,0.75,0.5,0.5,0.0,0.333333,0.197257,0.666667,0.000000,0.064019,0.182139,1.0,1.00,1.0,0.666667,0.326139,0.376477,0.0,0.548792,0.0,0.0,0.50,0.5,0.500000,0.5,0.25,0.250000,0.75,0.25,1.0,0.5,0.678344,0.333333,0.4,0.323925,0.666667,1.0,1.0,0.148876,0.045822,0.0,0.0,0.000000,0.0,1.0,0.50,0.5,0.000000,0.181818,1.0,0.5,0.5
3,0.235294,0.75,0.582212,0.524583,0.0,0.5,0.333333,1.000000,1.0,0.25,0.5,0.95,0.333333,0.0,0.0,0.4,0.555556,0.625,0.100000,0.225806,0.0,0.0,0.5,0.363636,0.25,0.015504,1.0,0.666667,0.75,1.0,0.5,0.0,0.333333,0.150125,0.666667,0.000000,0.151402,0.181747,1.0,0.75,1.0,0.666667,0.325285,0.364125,0.0,0.542672,0.0,0.0,0.50,0.5,0.500000,0.5,0.50,0.333333,0.75,0.25,0.4,0.5,0.675159,0.333333,0.4,0.315860,0.666667,1.0,1.0,0.252809,0.048518,0.0,0.0,0.000000,0.0,1.0,0.25,0.5,0.000000,0.454545,1.0,0.5,0.5
4,0.588235,0.75,0.317987,0.335596,0.0,0.5,0.333333,0.666667,1.0,0.25,0.5,1.00,0.333333,0.0,0.5,0.6,0.777778,0.500,0.146154,0.322581,0.0,0.0,0.9,0.727273,0.50,0.000000,0.0,0.666667,0.75,0.5,0.5,0.0,0.833333,0.065586,0.666667,0.000000,0.475234,0.251227,1.0,0.75,1.0,0.666667,0.453388,0.000000,0.0,0.453388,0.0,0.0,0.50,0.0,0.333333,0.5,0.50,0.166667,0.75,0.00,0.2,0.5,0.694268,1.000000,0.4,0.340054,0.666667,1.0,1.0,0.000000,0.110512,0.0,0.0,0.250000,0.0,1.0,0.25,0.5,0.000000,0.000000,1.0,0.5,0.5


In [11]:
nan_features1 = [feature for feature in X_train.columns if dataset_train[feature].isnull().sum()>1]
nan_values1 = dataset_train[nan_features1].isnull().sum()
nan_features1
# nan_values1

[]

In [23]:
select_features = SelectFromModel(Lasso(alpha = 0.005, random_state = 0))
nan_features2 = [feature for feature in X_train.columns if dataset_train[feature].isnull().sum()>1]
nan_values2 = dataset_train[nan_features2].isnull().sum()
nan_features2


[]

In [24]:
select_features.fit(X_train, Y_train)

ValueError: Input X contains NaN.
Lasso does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [25]:
select_features.get_support()

ValueError: when `importance_getter=='auto'`, the underlying estimator Lasso should have `coef_` or `feature_importances_` attribute. Either pass a fitted estimator to feature selector or call fit before calling transform.

In [ ]:
selected_features = X_train.columns[(select_features.get_support())]

In [ ]:
print("Total number of selected feature:", len(selected_features))
print("Total numner of feature:", X_train.shape[1])
print("Feature with coefficients shrank to zero:", np.sum(select_features.estimator_.coef_ == 0) )

In [ ]:
selected_features

In [ ]:
X_train = X_train[selected_features]

In [ ]:
X_train.head()